## 뉴스 크롤링하기

#### 삼성전자 검색후 title, link

In [ ]:
import requests
from bs4 import BeautifulSoup

resp = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90")
html = resp.text
soup = BeautifulSoup(html,'html.parser')
links = soup.select(".news_tit")
# print(links)

for link in links:
    title = link.text # 태크안에 텍스트요소 가져옴 
    url = link.attrs['href'] # href속성 가져옴
    print(title, url)

# Protocol://Domain/Path?Parameter


#### 검색어 입력받고 1~10 page title, link

In [ ]:
import requests
from bs4 import BeautifulSoup
import pyautogui
num = 1
# keyword = input('검색어를 입력하세요>>')
keyword = pyautogui.prompt('검색어를 입력하세요.')

for i in range(1,101,10):
    print(f"{num}page입니다@@@@@@@@@@@@@@@@@@@@")
    resp = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start{i}")
    html = resp.text
    soup = BeautifulSoup(html,'html.parser')
    links = soup.select(".news_tit")
# print(links)
    for link in links:
        title = link.text # 태크안에 텍스트요소 가져옴 
        url = link.attrs['href'] # href속성 가져옴
        print(title, url)
    num += 1

#### 네이버 뉴스 본문 크롤링

왜 네이버 뉴스만 크롤링?
네이버 뉴스의 본문은 전부 생김새가 동일하나 다른 뉴스들은 생김새가 달라 크롤링이 어렵다

##### 네이버 뉴스 링크 가져오는 전략
1. div.info_group 아래에 a.info가 2개이상인것만
2. 2번째 a.info의 href속성값 가져온다

#### 페이지내에 링크만 10개 가져올때

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pyautogui



resp = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90')
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
links = soup.select("div.info_group")

for link in links:
    a_tags = link.find_all('a', href=True)
    if len(a_tags) >= 2:
        url = a_tags[1]['href']
        print(url)

#### 본문 크롤링할때

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

resp = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90')
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info") # a tag에 클래스는 info
    if len(links) >= 2:
        url = links[1].attrs['href']
        # print(url)
        resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 각링크들 받아온후 
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        # print(soup)
        contents = soup.select_one('#dic_area') # dic_area: 본문 뉴스 id(#) OR newsct_body: 본문 뉴스 class(.)
        if contents:  # 결과가 있을 경우만 출력
            print(contents.text)
        time.sleep(0.5) 

### 검색어 입력받고, 해당 (네이버)뉴스들 출력하기 

In [13]:
import requests
from bs4 import BeautifulSoup
import time

# 사용자로부터 검색어를 입력받습니다.
query = input("검색어를 입력하세요: ")

base_url = 'https://search.naver.com/search.naver'
params = {
    'where': 'news',
    'sm': 'tab_jum',
    'query': query
}

resp = requests.get(base_url, params=params)
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info") # a tag에 클래스는 info
    if len(links) >= 2:
        url = links[1].attrs['href']
        resp = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        print(url)
        contents = soup.select_one('#dic_area')
        if contents: 
            print(contents.text)
        time.sleep(0.5)


### 연예 뉴스 크롤링하기 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

resp = requests.get('https://search.naver.com/search.naver?where=news&sm=tab_jum&query=%EC%97%90%EC%8A%A4%ED%8C%8C') # 에스파 검색링크
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info") # a tag에 클래스는 info
    if len(links) >= 2:
        url = links[1].attrs['href']
        # print(url)
        resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 각링크들 받아온후 
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        print(url)

        # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
        if "entertain" in resp.url:
            title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
            contents = soup.select_one("#articeBody") # 연예 뉴스 본문
        else:
            title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
            contents = soup.select_one('#dic_area') # 일반 뉴스 본문
        print("==========링크===========\n",url)
        print("==========제목===========\n",title.text.strip())
        print("==========본문===========\n",contents.text.strip())
        time.sleep(0.5) # 필수!!!!!

#### 스포츠뉴스는 사이트의 생김새(HTML)가 다르기때문에 오류가 난다
### 해결법 
- 본문 사이트 요청시 사이트 URL이 스포츠뉴스라면? 
- CSS 선택자를 바꿔준다

#### if 연예뉴스 URL 
     연예뉴스 사이트의 CSS 선택자 사용
#### elif 스포츠뉴스 URL
     스포츠 뉴스 사이트의 CSS 선택자 사용
#### else 
     일반 뉴스 사이트의 CSS 선택자 사용

In [ ]:
import requests
from bs4 import BeautifulSoup
import time

resp = requests.get('https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query=%EC%98%A4%EC%8A%B9%ED%99%98+&oquery=%EC%86%90%ED%9D%A5%EB%AF%BC&tqi=iL45NlprvTosshov5lVssssstdZ-295557') # 에스파 검색링크
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info") # a tag에 클래스는 info
    if len(links) >= 2:
        url = links[1].attrs['href']
        # print(url)
        resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 각링크들 받아온후 
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        print(url)

        # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
        if "entertain" in resp.url:
             title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
             contents = soup.select_one("#articeBody") # 연예 뉴스 본문

        elif "sports" in resp.url:
                title = soup.select_one("h4.title")
                contents = soup.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기
                divs = contents.select('div')
                for div in divs:
                    div.decompose()  # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
        else:
                title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup.select_one('#dic_area') # 일반 뉴스 본문

        print("==========링크===========\n",url)
        print("==========제목===========\n",title.text.strip())
        print("==========본문===========\n",contents.text.strip())
        time.sleep(0.5) # 필수!!!!!

### 검색어 입력받고 해당 (네이버)뉴스 본문들 출력하기

In [ ]:
import requests
from bs4 import BeautifulSoup
import pyautogui
keyword = pyautogui.prompt('검색어를 입력하세요.')
resp = requests.get(
    f'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={keyword}')
html = resp.text
soup = BeautifulSoup(html, 'html.parser')
articles = soup.select("div.info_group")

for article in articles:
    links = article.select("a.info")  # a tag에 클래스는 info
    if len(links) >= 2:
        url = links[1].attrs['href']
        # print(url)
        resp = requests.get(
            url, headers={'User-agent': 'Mozila/5.0'})  # 각링크들 받아온후
        html = resp.text
        soup = BeautifulSoup(html, 'html.parser')
        print(url)
        # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
        if "entertain" in resp.url:
             title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
             contents = soup.select_one("#articeBody") # 연예 뉴스 본문

        elif "sports" in resp.url:
                title = soup.select_one("h4.title")
                contents = soup.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기
                divs = contents.select('div')
                for div in divs:
                    div.decompose()  # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
        else:
                title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup.select_one('#dic_area') # 일반 뉴스 본문

        print("==========링크===========\n", url)
        print("==========제목===========\n", title.text.strip())
        print("==========본문===========\n", contents.text.strip())
        time.sleep(0.5)  # 필수!!!!!


### 사용자로부터 페이지 수 입력받고 그 페이지 수만큼 크롤링

In [ ]:

import requests
from bs4 import BeautifulSoup
import pyautogui

keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?")) # int변환 필요

page_num = 1
for i in range(1, lastpage * 10, 10):
    print(f"{page_num} 페이지 크롤링 중 입니다.======================================")
    resp = requests.get(f'https://search.naver.com/search.naver?sm=tab_hty.top&where=news&query={keyword}start={i}')
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info") # a tag에 클래스는 info
        if len(links) >= 2:
            url = links[1].attrs['href']
            # print(url)
            resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 각링크들 받아온후 
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')
            print(url)

                       # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
            if "entertain" in resp.url:
                title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
                contents = soup.select_one("#articeBody") # 연예 뉴스 본문

            elif "sports" in resp.url:
                title = soup.select_one("h4.title")   
                contents = soup.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기 
                divs = contents.select('div')
                for div in divs:
                    div.decompose() # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
            else:
                title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup.select_one('#dic_area') # 일반 뉴스 본문

            print("==========링크===========\n",url)
            print("==========제목===========\n",title.text.strip())
            print("==========본문===========\n",contents.text.strip())
            time.sleep(0.5) # 필수!!!!!
    page_num += 1


### 워드 문서 다루기 

In [7]:
from docx import Document

#1. 워드 생성하기 
document = Document()

#2. 워드 데이터 추가하기 
document.add_heading('기사제목', level=0)
document.add_paragraph('기사 링크')
document.add_paragraph('기사 본문')

#3. 워드 저장하기
document.save("text.docx")

### 크롤링 결과를 워드파일에 저장해 보자!
파일 생성 위치, 데이터 입력 위치, 파일 저장 위치 고려하기

In [16]:

import requests
from bs4 import BeautifulSoup
import pyautogui
from docx import Document

document = Document()

keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?")) # int변환 필요

page_num = 1
for i in range(1, lastpage * 10, 10):
    print(f"{page_num} 페이지 크롤링 중...")
    resp = requests.get(f'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}start={i}')
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info") # a tag에 클래스는 info
        if len(links) >= 2:
            url = links[1].attrs['href']
            # print(url)
            resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 각링크들 받아온후 
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')

            # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
            if "entertain" in resp.url:
                title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
                contents = soup.select_one("#articeBody") # 연예 뉴스 본문

            elif "sports" in resp.url:
                title = soup.select_one("h4.title")   
                contents = soup.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기 
                divs = contents.select('div')
                for div in divs:
                    div.decompose() # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
            else:
                title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup.select_one('#dic_area') # 일반 뉴스 본문

            document.add_heading(title.text.strip(), level=0)
            document.add_paragraph(url)
            document.add_paragraph(contents.text.strip())
            time.sleep(0.5) # 필수!!!!!
    page_num += 1
document.save(f"{keyword}_result.docx")


1 페이지 크롤링 중...
2 페이지 크롤링 중...
3 페이지 크롤링 중...


### 워드 다루기 

In [1]:
from openpyxl import Workbook

# 엑샐 생성하기 
wb = Workbook()

# 엑셀 시트 생성하기
ws = wb.create_sheet("이도형")
ws['A1'] = '이도형'

wb.save('DHL.xlsx')

In [9]:

import requests
from bs4 import BeautifulSoup
import pyautogui
from openpyxl import Workbook
from openpyxl.styles import Alignment
import time

keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?")) # int변환 필요

wb = Workbook()
ws = wb.create_sheet(keyword)
# 너비 설정 
ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 60
ws.column_dimensions['C'].width = 120

page_num = 1
row = 1

for i in range(1, lastpage * 10, 10):
    print(f"{page_num} 페이지 크롤링 중...")
    resp = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}")
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info") # a tag에 클래스는 info
        if len(links) >= 2:
            url = links[1].attrs['href']
            # print(url)
            resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 에러 방지
            html = resp.text
            soup = BeautifulSoup(html, 'html.parser')

            # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
            if "entertain" in resp.url:
                title = soup.select_one(".end_tit") # 연예 뉴스 타이틀 
                contents = soup.select_one("#articeBody") # 연예 뉴스 본문

            elif "sports" in resp.url:
                title = soup.select_one("h4.title")   
                contents = soup.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기 
                divs = contents.select('div')
                for div in divs:
                    div.decompose() # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
            else:
                title = soup.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup.select_one('#dic_area') # 일반 뉴스 본문

            ws[f'A{row}'] = url
            ws[f'B{row}'] = title.text.strip()
            ws[f'C{row}'] = contents.text.strip()
            ws[f"C{row}"].alignment = Alignment(wrap_text=True)  
            row += 1
            time.sleep(0.4)
    page_num += 1
    wb.save(f"{keyword}_result.xlsx")


1 페이지 크롤링 중...
2 페이지 크롤링 중...


### 마지막 페이지 확인하기

In [11]:

import requests
from bs4 import BeautifulSoup
import pyautogui
from openpyxl import Workbook
from openpyxl.styles import Alignment
import time

keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?")) # int변환 필요

wb = Workbook()
ws = wb.create_sheet(keyword)
# 너비 설정 
ws.column_dimensions['A'].width = 60
ws.column_dimensions['B'].width = 60
ws.column_dimensions['C'].width = 120

page_num = 1
row = 1

for i in range(1, lastpage * 10, 10):
    print(f"{page_num} 페이지 크롤링 중...")
    resp = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}")
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info") # a tag에 클래스는 info
        if len(links) >= 2:
            url = links[1].attrs['href']
            # print(url)
            resp = requests.get(url, headers={'User-agent': 'Mozila/5.0'}) # 에러 방지
            html = resp.text
            soup_sub = BeautifulSoup(html, 'html.parser')

            # 만약 연예 뉴스라면? redirection이 발생하므로 url이 아닌 resp.url
            if "entertain" in resp.url:
                title = soup_sub.select_one(".end_tit") # 연예 뉴스 타이틀 
                contents = soup_sub.select_one("#articeBody") # 연예 뉴스 본문

            elif "sports" in resp.url:
                title = soup_sub.select_one("h4.title")   
                contents = soup_sub.select_one('#newsEndContents')

                    # contents중에서 불필요한 부분(기자명, 다른광고) div 삭제하기 
                divs = contents.select('div')
                for div in divs:
                    div.decompose() # 삭제
                paragraphs = contents.select('p')
                for p in paragraphs:
                    p.decompose()
            else:
                title = soup_sub.select_one(".media_end_head_title") # 일반 뉴스 타이틀 
                contents = soup_sub.select_one('#dic_area') # 일반 뉴스 본문

            ws[f'A{row}'] = url
            ws[f'B{row}'] = title.text.strip()
            ws[f'C{row}'] = contents.text.strip()
            ws[f"C{row}"].alignment = Alignment(wrap_text=True)  
            row += 1
            time.sleep(0.4)

    isLastPage = soup.select_one('a.btn_next').attrs['aria-disabled']
    if isLastPage == 'true':
        print("마지막 페이지입니다.")
        break 
    page_num += 1
    wb.save(f"{keyword}_result.xlsx")


1 페이지 크롤링 중...
2 페이지 크롤링 중...
3 페이지 크롤링 중...
마지막 페이지입니다.


## 워드 클라우드
#### 중요한 단어를 더 잘보이게 하는방법
- 본문 내용 합치기 -> 워드클라우드 만들기 

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import pyautogui
import pyperclip

keyword = pyautogui.prompt("검색어를 입력하세요")
lastpage = int(pyautogui.prompt("몇 페이지까지 크롤링 할까요?")) # int변환 필요

page_num = 1
total_content = ""
article_num = 0

for i in range(1, lastpage * 10, 10):
    print(f"{page_num} page...")
    response = requests.get(f"https://search.naver.com/search.naver?where=news&sm=tab_jum&query={keyword}&start={i}")
    html = response.text
    soup = BeautifulSoup(html, "html.parser")
    articles = soup.select("div.info_group")

    for article in articles:
        links = article.select("a.info")
        if len(links) >= 2:
            url = links[1].attrs["href"]
            response = requests.get(url, headers={'User-agent': 'Mozila/5.0'})    
            html = response.text                                                 
            soup_sub = BeautifulSoup(html, "html.parser")                         

            # separation
            if "entertain" in response.url:                                     
                content = soup_sub.select_one("#articeBody")                        
            elif "sports" in response.url:                                        
                content = soup_sub.select_one("#newsEndContents")             

                # delete unnecessary elements
                divs = content.select("div")
                for div in divs:
                    div.decompose()

                paragraphs = content.select("p")
                for p in paragraphs:
                    p.decompose()

            else:
                content = soup_sub.select_one("#dic_area")                         
            print("========BODY========\n", content.text.strip())
            total_content += content.text.strip()
            article_num += 1
            time.sleep(0.3)

    is_last_page = soup.select_one("a.btn_next").attrs["aria-disabled"]
    if is_last_page == "true":
        print("last page")
        break

    page_num += 1

print(f"{article_num} articles 크롤링 완료")
pyperclip.copy(total_content)
pyautogui.alert('클립보드에 복사되었습니다.')
